In [ ]:
# ======================================================
# SECTION 2 — STRATEGY LOGIC IMPLEMENTATION
# ======================================================

df = data.copy()
df = df.dropna().reset_index(drop=True)

# ==============================
# Z-SCORE CALCULATION
# ==============================
lookback = 120
df["spread_mean"] = df["spread"].rolling(lookback).mean()
df["spread_std"]  = df["spread"].rolling(lookback).std()

df["zscore"] = (df["spread"] - df["spread_mean"]) / df["spread_std"]

# ==============================
# TRADING PARAMETERS
# ==============================
entry_z = 2
exit_z = 0.5
stop_z = 3
max_hold_days = 25        # ~= 2 × Half Life
cost_per_trade = 5        # Adjust if needed

df["position"] = 0
df["entry_price"] = np.nan
df["days_in_trade"] = 0

positions = []
in_position = False
entry_index = None

# ==============================
# TRADE LOGIC
# ==============================
for i in range(len(df)):

    z = df.loc[i, "zscore"]

    # ================= ENTRY RULES =================
    if not in_position:

        if z <= -entry_z:
            in_position = True
            side = 1   # Long Spread
            entry_index = i
            df.loc[i, "position"] = side

        elif z >= entry_z:
            in_position = True
            side = -1  # Short Spread
            entry_index = i
            df.loc[i, "position"] = side

    # ================= EXIT RULES =================
    else:
        df.loc[i, "position"] = side
        holding_days = i - entry_index

        exit_signal = (
            abs(z) < exit_z or
            abs(z) > stop_z or
            holding_days >= max_hold_days
        )

        if exit_signal:
            in_position = False
            
            entry = df.loc[entry_index, "spread"]
            exitp = df.loc[i, "spread"]

            if side == 1:   # Long spread
                pnl = exitp - entry
            else:           # Short spread
                pnl = entry - exitp

            pnl -= cost_per_trade

            positions.append({
                "entry_time": df.loc[entry_index, "TIMESTAMP"],
                "exit_time":  df.loc[i, "TIMESTAMP"],
                "entry_spread": entry,
                "exit_spread": exitp,
                "holding_days": holding_days,
                "trade_pnl": pnl,
                "direction": "LONG" if side==1 else "SHORT"
            })

# ==============================
# RESULTS DATAFRAME
# ==============================
trade_df = pd.DataFrame(positions)

print("Total Trades:", len(trade_df))
print(trade_df.head())

# ==============================
# STRATEGY RET SERIES
# ==============================
df["strategy_ret"] = 0

for _, t in trade_df.iterrows():
    mask = (df["TIMESTAMP"] >= t["entry_time"]) & (df["TIMESTAMP"] <= t["exit_time"])
    df.loc[mask, "strategy_ret"] = t["trade_pnl"] / len(df.loc[mask])

print("Section-2 Completed ✓ Strategy Built")
